# Step 1: List all pools

In [1]:
# 📊 Data Handling
import pandas as pd
import numpy as np
import requests
from flipside import Flipside
from datetime import datetime


# 📈 Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# ⚙️ Preprocessing & Scaling
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 📈 Analysis
import scipy.stats as stats
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
  
# 🧠 Modeling
from sklearn.cluster import KMeans, DBSCAN
import hdbscan
from sklearn.metrics import silhouette_score

In [11]:
# the list of dex we would like to see on dashboard
df['PLATFORM'].value_counts()

uniswap-v2        45398
uniswap-v3         7748
uniswap-v4         3908
sushiswap          1874
dodo-v2             782
balancer            476
pancakeswap-v2      438
shibaswap           418
curve               357
hashflow-v3         353
pancakeswap-v3      260
woofi               137
kyberswap-v1         27
verse                22
kyberswap-v2         17
fraxswap             11
dodo-v1              10
trader-joe-v2         1
maverick              1
Name: PLATFORM, dtype: int64

# Step 0: Based on dex -> indentify pools

In [2]:
# first we need to have a complete list of pools in specific dex

url = "https://yields.llama.fi/pools"
response = requests.get(url)
all_pools = response.json()['data']

curve_ethereum_pools = [
    pool for pool in all_pools
    if pool['project'] == 'curve-dex' and pool['chain'] == 'Ethereum'
]

df_curve_eth = pd.DataFrame([{
    'chain': pool['chain'],
    'project': pool['project'],
    'pool_name': pool.get('symbol', ''),
    'pool_id': pool['pool'],
    'stablecoin' : pool['stablecoin'],
    'tvlUsd' : pool['tvlUsd']

} for pool in curve_ethereum_pools])

In [3]:
def format_number(n):
    if n >= 1e9:
        return f"{n/1e9:.2f}B"
    elif n >= 1e6:
        return f"{n/1e6:.2f}M"
    elif n >= 1e3:
        return f"{n/1e3:.2f}K"
    else:
        return str(n)

df_curve_eth['tvlUsd_pretty'] = df_curve_eth['tvlUsd'].apply(format_number)

In [4]:
df_curve_eth.head(2)

,chain,project,pool_name,pool_id,stablecoin,tvlUsd,tvlUsd_pretty
0,Ethereum,curve-dex,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,True,182497692,182.50M
1,Ethereum,curve-dex,ETH-STETH,57d30b9c-fc66-4ac2-b666-69ad5f410cce,False,131390193,131.39M


# Chart 00: Market Share

In [5]:
# Filter by stable coins and converting tvl numbers
df_c = df_curve_eth[df_curve_eth['stablecoin']==True]
df_c1 = df_c.groupby(['pool_name','pool_id'])['tvlUsd'].mean().reset_index().sort_values(by='tvlUsd', ascending=False )
df_c1['tvlUsd_pretty'] = df_c1['tvlUsd'].apply(format_number)
df_c1['share'] = round((df_c1['tvlUsd'] / df_c1['tvlUsd'].sum()) * 100)

In [7]:
# pull out for chart 1
#df_result.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/onchain-analytics/01_Market_Making/df_tvl.csv')
df_c1[df_c1['share'] > 0]

,pool_name,pool_id,tvlUsd,tvlUsd_pretty,share
14,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,182497692.0,182.50M,20.0
42,FRAX-USDE,12ca9565-0369-404e-b209-631305e4012a,67930558.0,67.93M,7.0
79,USD0-USD0++,bc58a174-05f8-4e41-8f77-be9c13a614bf,67587904.0,67.59M,7.0
89,USDC-RLUSD,e91e23af-9099-45d9-8ba5-ea5b4638e453,51709437.0,51.71M,6.0
78,SUSDS-USDT,6f74baf2-48f3-4e4a-ad67-5c056f37e4fa,50650226.0,50.65M,5.0
25,DOLA-USR,d78803f4-437f-4ce5-b199-1bae142a185d,40145795.0,40.15M,4.0
13,CUSDO-USDC,08d1f13b-5d65-42c5-863d-fcb447dcea75,37137081.0,37.14M,4.0
87,USDC-FXUSD,bcc046f9-d980-40d9-80b2-5d5ae5931131,24075919.0,24.08M,3.0
70,SDAI-SUSDE,d22deafb-dbb1-417c-b555-cc8c9f4b955a,22067808.0,22.07M,2.0
100,USDL-USDC,b92d0d7b-6113-4981-9fa7-fad4d20cad6b,20947559.0,20.95M,2.0


# Chart 1: TVL Over The Time

In [12]:
import requests

# URL API
url = "https://yields.llama.fi/pools"

# گرفتن داده‌ها
response = requests.get(url)
data = response.json()["data"]

# استخراج projectهایی که chain شون Ethereum هست
ethereum_projects = {pool["project"] for pool in data if pool["chain"] == "Ethereum"}

# چاپ نتیجه
print("📌 Projects on Ethereum:")
for project in sorted(ethereum_projects):
    print("-", project)

📌 Projects on Ethereum:
- 1inch-network
- 3jane
- aave-v2
- aave-v3
- abracadabra-spell
- across
- aegis
- allbridge-classic
- amphor
- angle
- ankr
- anzen-v2
- arpa-staking
- arrakis-v1
- asymetrix-protocol
- aura
- badger-dao
- bakerfi
- balancer-v2
- balancer-v3
- bancor-v3
- bedrock-unieth
- beefy
- benddao-lending
- bifrost-liquid-staking
- binance-staked-eth
- cat-in-a-box
- cian-yield-layer
- clearpool-lending
- clipper
- coinbase-wrapped-staked-eth
- compound-v2
- compound-v3
- concentrator
- conic-finance
- convex-finance
- cream-lending
- crvusd
- curve-dex
- curve-llamalend
- damm-finance
- dinero-(pxeth)
- drops
- dsf.finance
- ethena-usde
- ether.fi-stake
- euler-v2
- fluid-lending
- fluid-lite
- flux-finance
- folks-finance-xchain
- fractal-protocol
- frax
- frax-ether
- fraxlend
- frigg.eco
- fringe-v1
- fx-protocol
- gamma
- gammaswap
- gearbox
- geth
- goldfinch
- gravita-protocol
- harmonix-finance
- harvest-finance
- hord
- hyperdrive
- impermax-v2
- index-coop
- in

In [13]:
import requests
import pandas as pd

# آدرس API
url = "https://yields.llama.fi/pools"

# دریافت داده‌ها
response = requests.get(url)
data = response.json()["data"]

# فیلتر کردن: فقط استخرهای curve-dex و sushiswap روی Ethereum که stablecoin هستند
filtered = [
    pool for pool in data
    if pool["chain"] == "Ethereum"
    and pool["project"] in ["curve-dex", "sushiswap",
                            "balancer-v2","balancer-v3",
                            "uniswap-v2","uniswap-v3","uniswap-v4",
                           "pancakeswap-amm","pancakeswap-amm-v3"]
    and pool.get("stablecoin", False) == True
]

# تبدیل به DataFrame
df = pd.DataFrame([{
    "project": pool["project"],
    "chain": pool["chain"],
    "pool_name": pool.get("symbol", ""),
    "pool_id": pool.get("pool", ""),
    "tvlUsd": pool.get("tvlUsd", 0)
} for pool in filtered])



In [14]:
df

,project,chain,pool_name,pool_id,tvlUsd
0,curve-dex,Ethereum,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,182014771
1,curve-dex,Ethereum,FRAX-USDE,12ca9565-0369-404e-b209-631305e4012a,67896126
2,curve-dex,Ethereum,USD0-USD0++,bc58a174-05f8-4e41-8f77-be9c13a614bf,67313461
3,uniswap-v3,Ethereum,DAI-USDC,1193ef25-862b-43c1-a545-91bbb9678d30,57282731
4,curve-dex,Ethereum,USDC-RLUSD,e91e23af-9099-45d9-8ba5-ea5b4638e453,52060334
...,...,...,...,...,...
203,curve-dex,Ethereum,FRAX-PYUSD,d049c13b-0073-44ce-ac92-649d7e64ac04,11322
204,curve-dex,Ethereum,FRAX-USDP,ea49024e-942d-4e68-987a-9bd4bbe124c5,11314
205,curve-dex,Ethereum,APEUSD-FRAX,5a2cac97-893f-4124-a15f-e5171cce21f5,10938
206,curve-dex,Ethereum,REUSD-USDC,3c7d73df-4886-4184-9d40-d60c09f4d289,10068


In [4]:
# Input pool id based on privouse table

pool_info = {
    "25171c4c-1877-449a-9f88-45a9f153ee31": "DAI-USDC-USDT"
}

start_date = datetime(2025, 5, 12)
end_date = datetime(2025, 6, 12)

results = []

for pool_id, pool_name in pool_info.items():
    url = f"https://yields.llama.fi/chart/{pool_id}"
    r = requests.get(url)
    if r.status_code != 200:
        print(f"❌ Failed for pool {pool_id}")
        continue

    data = r.json().get("data", [])

    for entry in data:
        ts = datetime.fromisoformat(entry["timestamp"].replace("Z", "")).replace(tzinfo=None)
        if start_date <= ts <= end_date:
            results.append({
                "pool_name": pool_name,
                "pool_id": pool_id,
                "timestamp": ts.date(), 
                "tvlUsd": entry.get("tvlUsd", None)
            })

df_result = pd.DataFrame(results)

In [6]:
# pull out for chart 2
#df_result.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/onchain-analytics/01_Market_Making/df_tvl.csv')
df_result[df_result['pool_name']=='FRAX-USDC']

,pool_name,pool_id,timestamp,tvlUsd
582,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-12,10718186
583,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-13,11148218
584,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-14,10850361
585,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-15,10749426
586,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-16,10671369
587,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-17,11030793
588,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-18,11533420
589,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-19,11038612
590,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-20,11425709
591,FRAX-USDC,3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1,2025-05-21,11742036


In [64]:
df_result['pool_name'].value_counts()

DAI-USDC-USDT         37
USD0-USD0++           37
FRAX-SDAI             37
DAI-USDC-USDT-SUSD    37
PYUSD-USDC            37
USDC-USDF             37
DOLA-SUSDS            37
USDC-FXUSD            37
DOLA-SUSDE            37
USDC-USDTB            37
USD0-USDC             37
USDL-USDC             37
SDAI-SUSDE            37
DOLA-USR              37
CUSDO-USDC            37
SUSDS-USDT            37
USDC-RLUSD            37
FRAX-USDE             37
FRAX-USDC             37
BOLD-USDC             19
Name: pool_name, dtype: int64

In [68]:


df_result[df_result['pool_name'] == "USDC-USDT"]


,pool_name,pool_id,timestamp,tvlUsd


In [67]:
df_curve_eth[df_curve_eth['pool_name'] == "USDC-USDT"]


,chain,project,pool_name,pool_id,stablecoin,tvlUsd,tvlUsd_pretty
79,Ethereum,curve-dex,USDC-USDT,e8dda16a-ad63-4925-b234-06861cff79c7,True,4900573,4.90M


# Chart 2: Slippage vs. Trade Size

In [ ]:
# just change the dex name

flipside = Flipside("bfb1c2a7-30de-46af-a084-5b8d9511280c", "https://api-v2.flipsidecrypto.xyz")

sql = """

SELECT
s.PLATFORM as "dex",
s.POOL_NAME as "pool_name",
s.SYMBOL_IN as "token_in",
s.SYMBOL_OUT as "token_out",
CAST(s.BLOCK_TIMESTAMP AS DATE) AS "timestamp" ,
round(avg(ABS(1 - (s.AMOUNT_OUT_USD / s.AMOUNT_IN_USD))) * 100,4) as "Spread",
count(s.TX_HASH) as "swap_count",
SUM(s.AMOUNT_IN_USD + s.AMOUNT_OUT_USD) as "volume",
avg(s.AMOUNT_IN_USD) as "order_size"
from ethereum.defi.ez_dex_swaps s
where s.BLOCK_TIMESTAMP between '2025-05-01' AND TIMESTAMP '2025-06-07'
 and s.AMOUNT_IN_USD > 100
and LOWER(s.PLATFORM) = 'curve'
group by 1,2,3,4,5

"""

query_result_set = flipside.query(sql)
df_c = pd.DataFrame(query_result_set.records)

In [69]:
df_c = pd.read_csv('df_main.csv')

In [70]:
# convert date
df_c['timestamp'] = pd.to_datetime(df_c['timestamp'], errors='coerce')
df_c['timestamp'] = df_c['timestamp'].dt.date
df_c['pool_name'] = df_c['token_in'] + "-" + df_c['token_out']

In [71]:
df_c.head()

,dex,pool,token_in,token_out,timestamp,Spread,swap_count,volume,order_size,pool_name
0,curve,WETH-CRV-crvUSD,WETH,crvUSD,2025-05-12,0.8117,103.0,772591.50,3752.408447,WETH-crvUSD
1,curve,OETH-WETH,OETH,WETH,2025-05-30,0.1215,60.0,817920.93,6819.052667,OETH-WETH
2,curve,Curve.fi Factory Plain Pool: msETH/WETH,WETH,msETH,2025-06-05,NaN,39.0,NaN,160670.506923,WETH-msETH
3,curve,WETH-CRV-crvUSD,WETH,crvUSD,2025-06-05,1.2249,101.0,732897.30,3655.466238,WETH-crvUSD
4,curve,crvFRAX-STG,STG,crvFRAX,2025-06-05,1.8009,6.0,47149.77,3971.651667,STG-crvFRAX


In [77]:
df_c.head()

,dex,pool,token_in,token_out,timestamp,Spread,swap_count,volume,order_size,pool_name
0,curve,WETH-CRV-crvUSD,WETH,crvUSD,2025-05-12,0.8117,103.0,772591.50,3752.408447,WETH-crvUSD
1,curve,OETH-WETH,OETH,WETH,2025-05-30,0.1215,60.0,817920.93,6819.052667,OETH-WETH
2,curve,Curve.fi Factory Plain Pool: msETH/WETH,WETH,msETH,2025-06-05,NaN,39.0,NaN,160670.506923,WETH-msETH
3,curve,WETH-CRV-crvUSD,WETH,crvUSD,2025-06-05,1.2249,101.0,732897.30,3655.466238,WETH-crvUSD
4,curve,crvFRAX-STG,STG,crvFRAX,2025-06-05,1.8009,6.0,47149.77,3971.651667,STG-crvFRAX


In [76]:
df_result.head()

,pool_name,pool_id,timestamp,tvlUsd
0,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-01,175508228
1,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-02,175492317
2,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-03,175114027
3,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-04,175365655
4,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,2025-05-05,175832607


In [78]:
df_curve_eth

,chain,project,pool_name,pool_id,stablecoin,tvlUsd,tvlUsd_pretty
0,Ethereum,curve-dex,DAI-USDC-USDT,25171c4c-1877-449a-9f88-45a9f153ee31,True,182497692,182.50M
1,Ethereum,curve-dex,ETH-STETH,57d30b9c-fc66-4ac2-b666-69ad5f410cce,False,131390193,131.39M
2,Ethereum,curve-dex,REUSD-SCRVUSD,5c4940c7-c193-440d-b95e-9148d017e12c,False,91558993,91.56M
3,Ethereum,curve-dex,ETH-OETH,18cb2d80-422e-4d0d-a223-66a307f06d8d,False,68205795,68.21M
4,Ethereum,curve-dex,FRAX-USDE,12ca9565-0369-404e-b209-631305e4012a,True,67930558,67.93M
...,...,...,...,...,...,...,...
397,Ethereum,curve-dex,FETH-FRAXBP,48fbcc9f-a12d-44ad-994a-caa81244bc17,False,10721,10.72K
398,Ethereum,curve-dex,REUSD-USDC,3c7d73df-4886-4184-9d40-d60c09f4d289,True,10067,10.07K
399,Ethereum,curve-dex,WA7A5-CRVUSD,258daf2d-2ccf-4f86-9c63-14a6a5dba998,False,10054,10.05K
400,Ethereum,curve-dex,USDB-USDC,5f996dd0-f86e-44ea-83c2-1fb0bdd421b4,True,10052,10.05K


In [80]:
df_c.to_csv('/Users/bahareh/Desktop/metrics.csv')